In [104]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import f1_score, precision_score, recall_score
from sklearn.metrics import precision_recall_fscore_support
#import import_ipynb
import utils_GTDmodel
from joblib import dump, load
#import imp
#imp.reload(utils_GTDmodel)
#from imblearn.combine import SMOTEENN

In [87]:
#DEFINE MAIN VARIABLES
filename = 'gtd_70to94_0617dist.xlsx'
var_name = '_70to94_03'
filename_data = 'data'+var_name+'.npz'
filename_one_hot = 'enc'+var_name+'.npz'
weight_type = '_balanced'
filename_tuning = 'tuning_cv'+weight_type+var_name+'.npz'
model_type = 'modelSVC_rbf'
C_best_type = '_C'
gamma_best_type = 'g'
features_dataset = ['iyear','imonth','iday','extended','crit1','crit2','crit3','doubtterr','country','region','attacktype1','success','weaptype1','targtype1']
features_class = ['gname']
n_folds = 5
scoring = ['precision_weighted', 'recall_weighted', 'f1_weighted', 'precision_micro', 'recall_micro', 'f1_micro']
Cs = [1, 10, 100, 1000]
gammas = [0.001, 0.0001]

In [4]:
#LOADING AND SAVE DATASET AND TARGET SET ON FILE
file_data = Path(filename_data)
if not file_data.is_file():
    saved_data_labels = utils_GTDmodel.dataset_creation(filename, filename_data, features_dataset, features_class)

#LOADING DATASET AND TARGTET SET FROM COMPRESSED DATA FILE
data = np.load(filename_data)
dataset = data[data.files[1]]
#dataset.shape (58099,14)
target = data[data.files[0]]
#target.shape (58099, 1)

In [5]:
#SPLITTING DATASET INTO DATASET FOR TRAINING AND DATASET FOR PREDICTION

#DATASET FOR PREDICTION: predict_data
predict_data_index = np.where(target=="Unknown")[0]
#predict_data_index.shape -> (18154,)
predict_data = dataset[predict_data_index,:]
#predict_data.shape -> (18154, 14)

In [6]:
#DATASET FOR TRAINING: X_dataset
X_dataset = np.delete(dataset, predict_data_index, 0)
#X_dataset.shape -> (39945, 14)
Y_target = np.delete(target, predict_data_index, 0)
#Y_target.shape -> (39945, 1)

In [7]:
#1.PREPROCESSING FEATURES: DELETE EXAMPLE WITH LESS THAN n_folds+2 EXAMPLES PER CLASS
info_y_target, count_y_target = np.unique(Y_target, return_counts=True)
#info_y_target.shape -> (1978, 1)
#1978 classes in the original dataset

for i in range(0, info_y_target.shape[0]):
    if count_y_target[i] < (n_folds+2):
        lessData_index = np.where(Y_target == info_y_target[i])[0]
        X_dataset = np.delete(X_dataset, lessData_index, 0)
        Y_target = np.delete(Y_target, lessData_index, 0)
#X_dataset.shape-> (37119, 14)
#Y_target.shape-> (37119, 1)

In [8]:
#2.PREPROCESSING FEATURES: FEATURES NORMALIZATION
X_norm_dataset = utils_GTDmodel.normalize_features(X_dataset)
#X_norm_dataset.shape-> (37119, 14)

feature 0
max: 1994
min: 1970
num_values: 24
[ 0  0  0 ..., 23 23 23]
[1970 1971 1972 1973 1974 1975 1976 1977 1978 1979 1980 1981 1982 1983 1984
 1985 1986 1987 1988 1989 1990 1991 1992 1994]


feature 1
max: 12
min: 0
num_values: 13
[ 0  1  1 ..., 12 12 12]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12]


feature 2
max: 31
min: 0
num_values: 32
[ 0  1  2 ..., 31 31 31]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31]


feature 3
max: 1
min: 0
num_values: 2
[0 0 0 ..., 0 0 0]
[0 1]


feature 4
max: 1
min: 0
num_values: 2
[1 1 1 ..., 1 1 1]
[0 1]


feature 5
max: 1
min: 0
num_values: 2
[1 1 1 ..., 1 1 1]
[0 1]


feature 6
max: 1
min: 0
num_values: 2
[1 1 1 ..., 1 1 1]
[0 1]


feature 7
max: 1
min: -9
num_values: 3
[1 1 1 ..., 1 0 1]
[-9  0  1]


feature 8
max: 605
min: 4
num_values: 133
[ 69 116 117 ..., 113  76 116]
[  4   6   7   8  11  12  14  15  16  18  19  20  21  26  28  29  30  32
  34  36  37  38  43  44  45  49  50  51  53  55  57  

In [9]:
#3. PREPROCESSING FEATURES: ONE HOT ENCODING USING X_norm_dataset
X_set = utils_GTDmodel.one_hot_encoding(filename_one_hot, X_norm_dataset)
print('encoding done')
print(X_set.shape)
print(list(X_set[0:1].toarray()))

#X_set.shape-> (37119, 268)

[ 24  13  32   2   2   2   2   3 133  12   9   2  10  22]
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 2

In [119]:
#RE-BALANCED DATSET USING IMBALANCED-LEARN
#print('plotting sparse matrix dataset')
#plt.figure(figsize=(14,12))
#plt.spy(X_set, aspect='auto')
print('starting resample')
Y_arr_target = [elem[0] for elem in Y_target]
smote_enn = SMOTEENN(random_state=42)
smote_enn.fit(X_set, Y_arr_target)
X_resampled, y_resampled = smote_enn.sample(X_set, Y_arr_target)
#print(sorted(Counter(y_resampled).items()))


starting resample


MemoryError: 

In [54]:
#TUNING HYPERPARAMETERS MODEL 

#X_train, X_test, y_train, y_test = train_test_split(X_set, Y_target, test_size=0.20)
Y_arr_target = [elem[0] for elem in Y_target]
X_train, X_test, y_train, y_test = train_test_split(X_set, Y_arr_target, test_size=0.20)
#X_train.shape -> (29695, 268)
#y_train.shape -> (29695L, 1L)
#X_test.shape -> (7424, 268)
#y_test.shape -> (7424L, 1L)

#BEST PARAMETERS
#class_weight='balanced': C = 1000; gamma = 0.001; best_f1 = 0.69223775  
file_tuning = Path(filename_tuning)
if not file_tuning.is_file():
    utils_GTDmodel.svc_param_selection(filename_tuning, Cs, gamma, weight_type, n_folds, X_train, y_train)
    
#PARAMETERS LOADING
tuning_results = np.load(filename_tuning)
f1_scores = tuning_results[tuning_results.files[0]]
C = np.float(tuning_results[tuning_results.files[1]])
gamma = np.float(tuning_results[tuning_results.files[2]])

In [105]:
#LEARNING MODEL
filename_model = model_type+C_best_type+str(int(C))+gamma_best_type+str(gamma)+weight_type+var_name+'.pkl'
file_model = Path(filename_model)
if not file_model.is_file():
    clf = SVC(kernel='rbf', C=C, gamma=gamma, class_weight=weight_type, verbose=10, decision_function_shape='ovr', random_state=42)
    clf.fit(X_train, y_train)
    #SAVE model
    dump(clf, filename_model)
else:
    clf = load(filename_model)

In [106]:
#TESTING MODEL
y_predicted = clf.predict(X_test)

#SCORES COMPUTATION
f1_micro = f1_score(y_test, y_predicted, average='micro')
recall_micro = recall_score(y_test, y_predicted, average='micro')
precision_micro = precision_score(y_test, y_predicted, average='micro')
# precision, recall, f1 and support for each label
labels_target = np.unique(Y_target)
scores_per_labels = precision_recall_fscore_support(y_test, y_predicted, average=None, labels=labels_target)

In [107]:
# SAVE SCORES INFO ON FILE
filename_test_scores = 'scores_'+model_type+C_best_type+str(int(C))+gamma_best_type+str(gamma)+weight_type+var_name+'.npz'
np.savez_compressed(filename_test_scores, f1_micro=f1_micro, precision_micro=precision_micro, recall_micro=recall_micro, labels_unique=labels_target, precision_labels=scores_per_labels[0], recall_labels=scores_per_labels[1], f1_labels=scores_per_labels[2], support_labels=scores_per_labels[3])
#scores_tot = np.load(filename_test_scores)
#scores_tot.files
##['recall_labels', 'f1_labels', 'support_labels', 'precision_labels', 'precision_micro', 'labels_unique', 'recall_micro', 'f1_micro']
#sl = scores_tot['support_labels']